In [2]:
from sklearn.datasets import fetch_20newsgroups
groups = fetch_20newsgroups()

In [3]:
data_train = fetch_20newsgroups(subset='train', random_state=21)
train_label = data_train.target
data_test = fetch_20newsgroups(subset='test', random_state=21)
test_label = data_test.target
len(data_train.data), len(data_test.data), len(test_label)

(11314, 7532, 7532)

In [4]:
import numpy as np
np.unique(test_label)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

## **Nettoyage du corpus et réduction du bruit dans ce dernier**

On filtre les éléments dans le corpus avec 2 conditions :

#### **1. Suppression des entités nommées**

Les prénoms peuvent introduire du bruit et biaiser l'apprentissage du modèle. Pour retirer les prénoms, on utilise le corpus NLTK qui contient de très nombreux noms en anglais. Ensuite on retire les noms dans la boucle, en utilisant `words not in all_names`.

#### **2. Suppression des caractères spéciaux**

On élimine la ponctuation (ex: "!", "?", ".") et les nombres avec `words.isalpha()`.


#### **Normalisation et lemmatisation**

Si un mot satisfait ces deux conditions, on applique `words.lower()` qui transforme le mot en minuscules et on réduit le mot à sa forme de base (lemme) avec `WNL.lemmatize(...)` . Cela réduit la dimensionalité du vocabulaire

In [5]:
import nltk
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names

nltk.download('names')
nltk.download('wordnet')
nltk.download('omw-1.4')  # OMW est l'acronyme de "Open Multilingual Wordnet"

all_names = set(names.words())
WNL = WordNetLemmatizer()

def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count += 1
    return list(cleaned.values())

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [6]:
x_train = clean(data_train.data)
x_train[0]

'bouncing lymenet lehigh university the following address are on the lymenet mailing but are rejecting since the list server originally accepted these address i assume these address have since been improperly functioning mail gateway might also be if you are listed here and would still like to remain on the please write to i will remove these address from the list before the next newsletter go a a general please remember to from all your mailing list before your account is this will save the listserv maintainer from many box lehigh university'

In [11]:
x_test = clean(data_test.data)
len(x_test)
len(x_train)

11314

In [8]:
# Décorateur chrono pour mesurer les temps d'exécution
import time

def chrono(f):
    def g(*a, **k):
        t = time.time()
        r = f(*a, **k)
        print(f"Temps : {time.time() - t:.4f} secondes")
        return r
    return g

# **SVM linéaire**


On distingue deux grands types de SVM linéaires : le **Hard margin** SVM et le **Soft margin** SVM.

### SVM **Hard Margin**

Dans le premier, les instances doivent être parfaitement séparables par un hyperplan linéaire.

Pour les points $x_i$ dans la classe $y_i = 1$ on impose :
$$\omega^\top x_i + b \ge 1$$

Pour les points $x_i$ dans la classe $y_i = -1$ on impose :
$$\omega^\top x_i + b \le -1$$

On constate qu'un élément $x_i$ est bien classifié si $y_i$ et $(\omega^\top x_i + b)$ ont le même signe. Ainsi, on peut reformuler ces deux contraintes de manière unifiée :
$$\boxed{\forall i,\quad y_i \cdot (\omega^\top x_i + b) \ge 1}$$


Mais cela suppose que les données soient parfaitement linéairement séparables.

Si ce n'est pas le cas, alors, quel que soit l’hyperplan choisi, il existera au moins un point $x_i$ pour lequel $y_i \cdot (\omega^\top x_i + b) \le 0$ c’est-à-dire au moins un point mal classé (ce qui arrive si $y_i$ et $(\omega^\top x_i + b)$ ont des signes différents), ou situé sur la frontière de décision (ce qui arrive si $y_i \cdot (\omega^\top x_i + b) = 0$).

Formellement, c'est linéairement séparable si

$$\exists (\omega,b)\ \text{tel que}\ \forall i,\ y_i(\omega^\top x_i + b) > 0$$


et non linéairement séparable si :

$$\forall (\omega,b),\ \exists i\ \text{tel que}\ y_i(\omega^\top x_i + b) \le 0$$

### SVM **Soft Margin**

C'est ce pourquoi on utilise un **SVM linéaire soft margin** qui fonctionne même si les données ne sont pas parfaitement linéairement séparables en tolérant des écarts et donc des erreurs de classifications éventuelles.

De manière générale, dans un SVM, il faut maximiser l'espacement entre les deux marges (qui correspondent aux deux hyperplans $\{ x \in \mathbb{R}^d \;|\; \omega^\top x + b = 1 \}$ et $\{ x \in \mathbb{R}^d \;|\; \omega^\top x + b = -1 \}$). Mais, avec certaines données non linéairement séparables, il faut trouver un compromis entre la largeur de la marge et les erreurs de classification.

Dans un SVM soft margin, on associe à chaque point $x_i$ une quantité $\xi_i$ qui indique “de combien il manque” pour satisfaire la condition idéale $y_i(\omega^\top x_i + b) \ge 1$. Pour chaque point $x_i$, la quantité $\xi_i$ est donnée par :

$$\xi_i = \max(0, 1 - y_i (\omega^\top x_i + b))$$

Il y a trois cas possibles (bien classé, dans la marge, mal classé) :

* Si le point $x_i$ est bien classé :

$$
y_i (\omega^\top x_i + b) \ge 1
\quad\Rightarrow\quad
\big[1 - y_i (\omega^\top x_i + b)\big] \le 0
\quad\Rightarrow\quad
\xi_i = 0
$$

* Si le point $x_i$ est dans la marge (entre les deux hyperplans) mais correctement classé :

$$
0 < y_i (\omega^\top x_i + b) < 1
\quad\Rightarrow\quad
0 < \xi_i = \big[1 - y_i (\omega^\top x_i + b)\big] < 1
$$

* Si le point $x_i$ est mal classé :

$$
y_i (\omega^\top x_i + b) < 0
\quad\Rightarrow\quad
\xi_i = \big[1 - y_i (\omega^\top x_i + b)\big] > 1
$$

Maintenant, on peut pénaliser le modèle à la fois en fonction du nombre de points du mauvais côté (en comptant à la fois les points dans les marges bien classés et ceux qui sont mal classés) et de l'ampleur des violations (plus la position d'un point s'écarte de sa position attendue, plus la pénalité sur la loss sera forte).

Pour ce faire, on dispose de l'hyperparamètre $C$ qui gère le compromis entre la largeur de la marge et les erreurs sur l’échantillon.

Si $C$ est grand, on cherche à réduire au maximum ces erreurs quitte à avoir une marge plus petite ; si $C$ est petit, on tolère davantage les violations pour favoriser une marge plus large. Dans ce qui suit, on cherche la valeur optimale du paramètre $C$ avec `Optuna`


### Sources :

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

https://en.wikipedia.org/wiki/Support_vector_machine#Linear_SVM

In [9]:
pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 20.8 MB/s eta 0:00:00


## **Le choix de l'optimiseur d'hyperparamètres : GridSearch ou Optuna ?**


### **GridSearch**
**[`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)** effectue une recherche ***exhaustive*** : il teste toutes les combinaisons possibles dans des grilles d'hyperparamètres prédéfinies (force brute).
Bien qu'on ait la garantie de trouver l'optimum global dans l'espace discret prédéfini, le coût computationnel croît exponentiellement avec la taille de l'espace des valeurs possibles.


### **Optuna**
**[`Optuna`](https://pypi.org/project/optuna/)** utilise une recherche ***bayésienne*** (algorithme TPE - Tree-structured Parzen Estimator) en échantillonnant l'espace des hyperparamètres et en privilégiant les régions prometteuses identifiées par les évaluations précédentes. Cela permet d'obtenir de bonnes solutions avec moins d'itérations. C'est pas absolument nécessaire ici mais dans certains cas où les espaces d'hyperparamètres à tester sont de très grande dimension et continus c'est plus rapide et efficace.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import optuna
import optuna.visualization as vis


def create_objective_large(X, y, cv=3):
    def objective(trial):
        max_features = trial.suggest_int('max_features', 8000, 26000, step=2000)
        ngram_max = trial.suggest_int('ngram_max', 1, 2)
        max_df = trial.suggest_float('max_df', 0.5, 1.0)
        C = trial.suggest_float('C', 0.1, 10, log=True)

        pipeline = Pipeline([
            ('tf_idf', TfidfVectorizer(
                stop_words='english',
                max_features=max_features,
                ngram_range=(1, ngram_max),
                max_df=max_df
            )),
            ('svc', LinearSVC(C=C, dual=True, max_iter=3000))
        ])

        score = cross_val_score(pipeline, X, y, cv=cv, n_jobs=1).mean()
        return score

    return objective

@chrono
def run_optuna_large(X, y, n_trials, cv=3):
    objective = create_objective_large(X, y, cv)
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, n_jobs=-1, show_progress_bar=True)
    return study

study_large = run_optuna_large(x_train, train_label, n_trials=50)


print(f"Meilleurs paramètres : {study_large.best_params}")
print(f"Meilleur score CV : {study_large.best_value:.4f}")

vis.plot_param_importances(study_large)
vis.plot_optimization_history(study_large)
vis.plot_slice(study_large, params=["max_features"])

[I 2025-12-12 08:49:23,343] A new study created in memory with name: no-name-995dffff-4737-4c09-8dc5-2cc77dbee0c6


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-12-12 08:49:40,709] Trial 0 finished with value: 0.8782931291144048 and parameters: {'max_features': 20000, 'ngram_max': 1, 'max_df': 0.6065537062173589, 'C': 0.9919800023622236}. Best is trial 0 with value: 0.8782931291144048.
[I 2025-12-12 08:49:41,620] Trial 1 finished with value: 0.8751996947177109 and parameters: {'max_features': 16000, 'ngram_max': 1, 'max_df': 0.5300758499320483, 'C': 1.5326481425380183}. Best is trial 0 with value: 0.8782931291144048.
[I 2025-12-12 08:50:15,374] Trial 2 finished with value: 0.8759951224948934 and parameters: {'max_features': 20000, 'ngram_max': 2, 'max_df': 0.9065974659443521, 'C': 0.4113302648904239}. Best is trial 0 with value: 0.8782931291144048.
[I 2025-12-12 08:50:15,897] Trial 3 finished with value: 0.8591134374731384 and parameters: {'max_features': 10000, 'ngram_max': 2, 'max_df': 0.7799715210622715, 'C': 3.6261105810836507}. Best is trial 0 with value: 0.8782931291144048.
[I 2025-12-12 08:50:30,724] Trial 4 finished with value: